In [359]:
# Create Machine Learning Model from stock metric data

# Import necessary libraries & modules


import pandas as pd
import csv
import math
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression


In [360]:
# Import .csv file 
# Create DataFrame from .csv file

file = pd.read_csv('/Users/garrettwankel/PycharmProjects/New/Final_df.csv')
df = pd.DataFrame(file)



In [361]:
# Examine DataFrame

df.head()



,,Sym,SP,50-Day MA,200-Day MA,52W Range,MC,P/B,Vol,Avg Vol (3m),Avg Vol (10-day),Sh Out,Float,Insiders,Institutions
0,08-07-2020,crbp,6.61,7.35,6.36,3.29 - 8.78,560.17M,24.43,"1,324,087",1.58M,1.84M,72.49M,71.03M,4.93%,41.18%
1,08-07-2020,ccxi,55.24,56.30,50.83,6.37 - 65.43,3.75B,62.41,"366,281",732.02k,624.86k,67.83M,43.42M,28.93%,67.08%
2,08-07-2020,gwph,109.37,129.58,113.69,67.98 - 172.12,3.81B,5.87,"2,374,243",339.43k,271.39k,31.08M,30.43M,1.12%,83.33%
3,08-07-2020,imvt,27.16,25.17,19.53,8.34 - 29.96,2.11B,22.46,"253,162",427.77k,236.06k,81.81M,25.48M,73.18%,21.10%
4,08-07-2020,alny,143.53,153.62,131.42,73.32 - 167.33,16.55B,14.08,"344,523",769.09k,470.09k,114.82M,113.87M,0.80%,94.74%


In [362]:
# Rename column(s)

df.rename(columns={' ': 'Date'})



,Date,Sym,SP,50-Day MA,200-Day MA,52W Range,MC,P/B,Vol,Avg Vol (3m),Avg Vol (10-day),Sh Out,Float,Insiders,Institutions
0,08-07-2020,crbp,6.61,7.3500,6.3600,3.29 - 8.78,560.17M,24.43,"1,324,087",1.58M,1.84M,72.49M,71.03M,4.93%,41.18%
1,08-07-2020,ccxi,55.24,56.3000,50.8300,6.37 - 65.43,3.75B,62.41,"366,281",732.02k,624.86k,67.83M,43.42M,28.93%,67.08%
2,08-07-2020,gwph,109.37,129.5800,113.6900,67.98 - 172.12,3.81B,5.87,"2,374,243",339.43k,271.39k,31.08M,30.43M,1.12%,83.33%
3,08-07-2020,imvt,27.16,25.1700,19.5300,8.34 - 29.96,2.11B,22.46,"253,162",427.77k,236.06k,81.81M,25.48M,73.18%,21.10%
4,08-07-2020,alny,143.53,153.6200,131.4200,73.32 - 167.33,16.55B,14.08,"344,523",769.09k,470.09k,114.82M,113.87M,0.80%,94.74%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,09-14-2020,imvt,38.42,30.1700,22.6200,8.34 - 38.90,3.04B,11.34,"371,662",540.55k,497.32k,81.81M,30.84M,73.18%,21.10%
86,09-15-2020,crbp,1.97,6.8603,6.4899,1.8100 - 9.7800,153.62M,3.17,"4,650,459",3.08M,17.88M,81.71M,73.07M,4.93%,41.18%
87,09-15-2020,ccxi,52.96,52.4900,52.0700,6.44 - 65.43,3.31B,7.86,"470,425",595.05k,565k,68.89M,47.82M,28.93%,67.08%
88,09-15-2020,gwph,107.13,111.9300,111.0500,67.98 - 141.98,2.99B,4.34,"501,889",369.1k,405.46k,31.14M,30.42M,1.12%,83.33%


In [363]:
# Examine datatypes to understand what needs to be converted to numerical values

df.dtypes



                     object
Sym                  object
SP                  float64
50-Day MA           float64
200-Day MA          float64
52W Range            object
MC                   object
P/B                 float64
Vol                  object
Avg Vol (3m)         object
Avg Vol (10-day)     object
Sh Out               object
Float                object
Insiders             object
Institutions         object
dtype: object

In [364]:
# Subset DataFrame for ML model implementation
# Reset index of Subset

df2 = df[df['Sym'] == 'ccxi']
df2.reset_index(inplace=True)



In [365]:
# Remove non-numerical characters from featured columns for uniformity
# Convert object .dtype to float

df2['Vol'] = df2.replace(',', '', regex=True)
df2['Avg Vol (10-day)'] = df2.replace('k', '', regex=True)
df2[['Vol', 'Avg Vol (10-day)']] = df2[['Vol', 'Avg Vol (10-day)']].astype(float)
df2.dtypes



/Users/garrettwankel/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/garrettwankel/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/garrettwankel/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

index                 int64
                     object
Sym                  object
SP                  float64
50-Day MA           float64
200-Day MA          float64
52W Range            object
MC                   object
P/B                 float64
Vol                 float64
Avg Vol (3m)         object
Avg Vol (10-day)    float64
Sh Out               object
Float                object
Insiders             object
Institutions         object
dtype: object

In [380]:
# Define features variables

X = df2[['50-Day MA', '200-Day MA', 'Vol', 'Avg Vol (10-day)']]

y = df2['SP']




In [381]:
y.shape, X.shape

((21,), (21, 4))

In [382]:
# Split dataset into train & test, specify test_size
X = np.array(X)
X = preprocessing.scale(X)
y = np.array(y)

train_X, test_x, train_y, test_y = train_test_split(X, y, test_size=.20)



In [383]:
# Create function to determine best 'max_leaf_nodes' value for each model
# Create for loop to iterate through list of possible values
# Print 'max_leaf_nodes' with corresponding 'mean_absolute_error'

def get_mae(max_leaf_nodes, train_X, test_x, train_y, test_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=10)
    model2 = RandomForestRegressor(max_leaf_nodes=max_leaf_nodes, random_state=10)
    model3 = LinearRegression()
    model.fit(train_X, train_y)
    model2.fit(train_X, train_y)
    model3.fit(train_X, train_y)
    preds1 = model.predict(test_x)
    preds2 = model2.predict(test_x)
    preds3 = model2.predict(test_x)
    mae1 = mean_absolute_error(preds1, test_y)
    mae2 = mean_absolute_error(preds2, test_y)
    mae3 = mean_absolute_error(preds3, test_y)
    return(mae1, mae2, mae3)

for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, test_x, train_y, test_y)
    print(max_leaf_nodes, my_mae)
    
    

5 (1.4720000000000013, 0.8121036587301503, 0.8121036587301503)
50 (1.1400000000000006, 0.6964600000000203, 0.6964600000000203)
500 (1.1400000000000006, 0.6964600000000203, 0.6964600000000203)
5000 (1.1400000000000006, 0.6964600000000203, 0.6964600000000203)


In [384]:
# Create models for predicting
# Train models

model = DecisionTreeRegressor(random_state=2, max_leaf_nodes=50)
model2 = RandomForestRegressor(random_state=2, max_leaf_nodes=50, n_estimators=200, criterion='mae')
model3 = LinearRegression()
model.fit(train_X,train_y)
model2.fit(train_X, train_y)
model3.fit(train_X, train_y)


LinearRegression()

In [385]:
# Make predictions

preds1 = model.predict(test_x)
preds2 = model2.predict(test_x)
preds3 = model3.predict(test_x)


In [386]:
# Score each model to determine it's accuracy based on the returned coeeficient

print(model.score(test_x, test_y))
print(model2.score(test_x, test_y))
print(model3.score(test_x, test_y))

0.7880544472241922
0.8906366219245514
0.6548490767823963


In [387]:
# Find the 'mean abosolute error' of each model
# ** Helpful to determine how accurate model is **

mae1 = mean_absolute_error(preds1, test_y)
mae2 = mean_absolute_error(preds2, test_y)
mae3 = mean_absolute_error(preds3, test_y)


In [388]:
# Print 'mean aboslute error'

print(mae1)
print(mae2)
print(mae3)


1.0399999999999991
0.6942800000000233
1.2738344881400352


In [389]:
# Print True values and corresponding predictions to compare models

test_y, preds1, preds2, preds3



(array([47.55, 52.86, 54.6 , 54.14, 52.81]),
 array([46.35, 52.05, 54.19, 52.2 , 53.65]),
 array([48.9284  , 52.719875, 53.707875, 53.296325, 53.027075]),
 array([50.31765929, 52.07086865, 53.56832914, 53.12160426, 53.5723152 ]))

In [391]:
score = cross_val_score(model2, X, y, cv=5)
score

array([ -0.17820262, -22.73063617,  -5.78192077,   0.53715124,
        -0.84918218])